# tensorflow-tutorial-i-recap

### Big idea: express a numeric computation as a graph.

- graph nodes are **operations** which have any number of inputs and outputs
- Graph edges are **tensors** which flow between nodes


![graph](https://github.com/rgtjf/ecnunlp-tensorflow-tutorial/raw/master/lecture2/figs/graph.jpg)

# $$h = ReLU(Wx + b)$$ 

- placeholder
- Variable
- Operation

```python
        # tf Graph Input
        x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
        y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

        # Set model weights
        W1 = tf.Variable(tf.zeros([784, 100]), name='W1')
        b1 = tf.Variable(tf.zeros([100]), name='b1')

        h = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
        
        W2 = tf.Variable(tf.zeros([100, 10]), name='W2')
        b2 = tf.Variable(tf.zeros([10]), name='b2')
        
        # Construct model
        pred = tf.nn.softmax(tf.matmul(h, W2) + b2) # Softmax

        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
        # Gradient Descent
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        
        # session
        sess = tf.Session()
        sess.run(tf.initialize_all_variables())
        for i in range(1000):
            batch_x, batch_label = data.next_batch()
            sess.run(train_step, feed_dict={x: batch_x, y: batch_label}
```

## Broadcast

```
A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 1
```

## Conv

```python
conv2d(
    input,
    filter,
    strides,
    padding,
    use_cudnn_on_gpu=None,
    data_format=None,
    name=None
)
```

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


# Parameters
learning_rate = 0.1
training_epochs = 25
batch_size = 100
display_step = 1


class Data(object):
    
    def __init__(self):
        mnist = input_data.read_data_sets("../MNIST_data", one_hot=True)
        
        self.mnist = mnist
        
    def batch_iter(self, batch_size):
        return  self.mnist.train.next_batch(batch_size)
    
    def test_data(self):
        x = self.mnist.test.images[:3000]
        y = self.mnist.test.labels[:3000]
        return x, y
    
    def __len__(self):
        return self.mnist.train.num_examples

    
        
class Model(object):
    
    def __init__(self):
        
        # tf Graph Input
        x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
        y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

        # Set model weights
        W1 = tf.Variable(tf.zeros([784, 100]), name='W1')
        b1 = tf.Variable(tf.zeros([100]), name='b1')

        h = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
        
        W2 = tf.Variable(tf.zeros([100, 10]), name='W2')
        b2 = tf.Variable(tf.zeros([10]), name='b2')
        
        # Construct model
        pred = tf.nn.softmax(tf.matmul(h, W2) + b2) # Softmax

        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
        # Gradient Descent
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        
        self.x = x
        self.y = y
        self.pred = pred
        self.cost = cost
        self.optimizer = optimizer
        
    
    def train_model(self, sess, batch_xs, batch_ys):
        feed_dict = {
            self.x: batch_xs,
            self.y: batch_ys
        }
        to_return = {
            'train_op': self.optimizer,
            'loss': self.cost
        }
        return sess.run(to_return, feed_dict)

    def test_model(self, sess, batch_xs):
        feed_dict = {
            self.x: batch_xs
        }
        to_return = {
            'pred': self.pred
        }
        return sess.run(to_return, feed_dict)
        

    def eval_model(self, sess, batch_xs, batch_ys):
         # Test model
        correct_prediction = tf.equal(tf.argmax(self.pred, 1), tf.argmax(self.y, 1))
        # Calculate accuracy for 3000 examples
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        feed_dict = {
            self.x: batch_xs,
            self.y: batch_ys
        }
        to_return = {
            'acc': accuracy
        }
        return sess.run(to_return, feed_dict)
        
data = Data()
model = Model()

# Start training
with tf.Session() as sess:
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(data) / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = data.batch_iter(batch_size)
            # Fit training using batch data
            results = model.train_model(sess, batch_xs, batch_ys)
            c = results['loss']
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            
    print("Optimization Finished!")

    test_xs, test_ys = data.test_data()
    acc = model.eval_model(sess, test_xs, test_ys)
    print("Accuracy:", acc)

## Recap
- data
    - [784] -> [10]

- model [LR/SVM/etc]
  - train
  - predict

- evaluation

## Recap (Cont.)

- Model

```python
# define the placeholder

# define the variables

# build the model graph
# including: predict, loss, and train_op
```

- Text Classification Task
  - [model](https://github.com/rgtjf/tf-classification/blob/master/src/models/NBoW.py)
  - [data](https://github.com/rgtjf/tf-classification/blob/master/src/data.py)
  - [main](https://github.com/rgtjf/tf-classification/blob/master/src/main.py)

- EASY to change to LSTM, HOW?
  - [model](https://github.com/rgtjf/tf-classification/blob/master/src/models/LSTM.py)